In [1]:
import sys,os,re
import time
import math
import string
import pandas as pd
pd.set_option('display.max_rows',1000000)
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\birha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#global variables used as counters

In [3]:
tokens=0
documents=0
terms=0
termindex=0
docindex=0


In [4]:
stop_words=stopwords.words('english')

In [5]:
#initialize list variables

In [6]:
alltokens=[]
alldocs=[]

In [7]:
#Capture the start time of the routine so that we can determine the total
#running time required to process the corpus

In [8]:
t2=time.localtime()

In [9]:
#set the name of the directory for the corpus

In [10]:
dirname="C:\\Users\\birha\\Documents\\cacm"

In [11]:
#for each document in the directory read the document into a string

In [12]:
all=[f for f in os.listdir(dirname)]
alldo={} #Dictionary for getting 
for f in all:
    documents+=1
    with open(dirname+'/'+f,'r') as myfile:
        alldocs.append(f)
       # alldo = {x: myfile.read().split() for x in all}
        data=myfile.read().replace('\n','')
        for token in data.split():
                alltokens.append(token)
                tokens+=1


# Creating Dictionary for the parased text

In [13]:
d = {}
for x in all:
    with open(dirname+'/'+x) as f:
        d[x] = f.read().replace('\t','').strip().split()


In [16]:
df=[pd.DataFrame(d[key]) for key in d.keys()]


In [25]:
#open for write a file for the document dictionary

In [17]:
documentfile=open(dirname+'/'+'documents.dat','w')
alldocs.sort()
for f in alldocs:
    docindex+=1
    documentfile.write(f+','+str(docindex)+os.linesep)
documentfile.close()


In [18]:
#sort the tokens in the list


In [19]:
alltokens.sort()

In [20]:
#identify unique terms in the corpus


In [21]:
g=[]

In [22]:
for i in alltokens:
    if i not in g:
        g.append(i)
        terms+=1
terms=len(g)

In [24]:
#output index to disk file. As part of this process we assign
# an index number to each unique term

In [28]:
indexfile=open(dirname+'/'+'index.dat','w')
for i in g:
    termindex +=1
    indexfile.write(i+','+str(termindex)+os.linesep)
indexfile.close()
    

# Creating function for stop words, punctuations and lower cases

In [29]:
def stop_word_removal(text, stop_word_corpus, punct_str):
    clean_text = ' '.join([word.lower() for word in text.split(',') if word.lower()
                 not in stop_word_corpus]).replace('\n',',')
    return clean_text.translate(str.maketrans('', '', punct_str))

In [30]:
class PorterStemmer:

    def __init__(self):
        """The main part of the stemming algorithm starts here.
        b is a buffer holding a word to be stemmed. The letters are in b[k0],
        b[k0+1] ... ending at b[k]. In fact k0 = 0 in this demo program. k is
        readjusted downwards as the stemming progresses. Zero termination is
        not in fact used in the algorithm.

        Note that only lower case sequences are stemmed. Forcing to lower case
        should be done before stem(...) is called.
        """

        self.b = ""  # buffer for word to be stemmed
        self.k = 0
        self.k0 = 0
        self.j = 0   # j is a general offset into the string

    def cons(self, i):
        """cons(i) is TRUE <=> b[i] is a consonant."""
        if self.b[i] == 'a' or self.b[i] == 'e' or self.b[i] == 'i' or self.b[i] == 'o' or self.b[i] == 'u':
            return 0
        if self.b[i] == 'y':
            if i == self.k0:
                return 1
            else:
                return (not self.cons(i - 1))
        return 1

    def m(self):
        """m() measures the number of consonant sequences between k0 and j.
        if c is a consonant sequence and v a vowel sequence, and <..>
        indicates arbitrary presence,

           <c><v>       gives 0
           <c>vc<v>     gives 1
           <c>vcvc<v>   gives 2
           <c>vcvcvc<v> gives 3
           ....
        """
        n = 0
        i = self.k0
        while 1:
            if i > self.j:
                return n
            if not self.cons(i):
                break
            i = i + 1
        i = i + 1
        while 1:
            while 1:
                if i > self.j:
                    return n
                if self.cons(i):
                    break
                i = i + 1
            i = i + 1
            n = n + 1
            while 1:
                if i > self.j:
                    return n
                if not self.cons(i):
                    break
                i = i + 1
            i = i + 1

    def vowelinstem(self):
        """vowelinstem() is TRUE <=> k0,...j contains a vowel"""
        for i in range(self.k0, self.j + 1):
            if not self.cons(i):
                return 1
        return 0

    def doublec(self, j):
        """doublec(j) is TRUE <=> j,(j-1) contain a double consonant."""
        if j < (self.k0 + 1):
            return 0
        if (self.b[j] != self.b[j-1]):
            return 0
        return self.cons(j)

    def cvc(self, i):
        """cvc(i) is TRUE <=> i-2,i-1,i has the form consonant - vowel - consonant
        and also if the second c is not w,x or y. this is used when trying to
        restore an e at the end of a short  e.g.

           cav(e), lov(e), hop(e), crim(e), but
           snow, box, tray.
        """
        if i < (self.k0 + 2) or not self.cons(i) or self.cons(i-1) or not self.cons(i-2):
            return 0
        ch = self.b[i]
        if ch == 'w' or ch == 'x' or ch == 'y':
            return 0
        return 1

    def ends(self, s):
        """ends(s) is TRUE <=> k0,...k ends with the string s."""
        length = len(s)
        if s[length - 1] != self.b[self.k]: # tiny speed-up
            return 0
        if length > (self.k - self.k0 + 1):
            return 0
        if self.b[self.k-length+1:self.k+1] != s:
            return 0
        self.j = self.k - length
        return 1

    def setto(self, s):
        """setto(s) sets (j+1),...k to the characters in the string s, readjusting k."""
        length = len(s)
        self.b = self.b[:self.j+1] + s + self.b[self.j+length+1:]
        self.k = self.j + length

    def r(self, s):
        """r(s) is used further down."""
        if self.m() > 0:
            self.setto(s)

    def step1ab(self):
        """step1ab() gets rid of plurals and -ed or -ing. e.g.

           caresses  ->  caress
           ponies    ->  poni
           ties      ->  ti
           caress    ->  caress
           cats      ->  cat

           feed      ->  feed
           agreed    ->  agree
           disabled  ->  disable

           matting   ->  mat
           mating    ->  mate
           meeting   ->  meet
           milling   ->  mill
           messing   ->  mess

           meetings  ->  meet
        """
        if self.b[self.k] == 's':
            if self.ends("sses"):
                self.k = self.k - 2
            elif self.ends("ies"):
                self.setto("i")
            elif self.b[self.k - 1] != 's':
                self.k = self.k - 1
        if self.ends("eed"):
            if self.m() > 0:
                self.k = self.k - 1
        elif (self.ends("ed") or self.ends("ing")) and self.vowelinstem():
            self.k = self.j
            if self.ends("at"):   self.setto("ate")
            elif self.ends("bl"): self.setto("ble")
            elif self.ends("iz"): self.setto("ize")
            elif self.doublec(self.k):
                self.k = self.k - 1
                ch = self.b[self.k]
                if ch == 'l' or ch == 's' or ch == 'z':
                    self.k = self.k + 1
            elif (self.m() == 1 and self.cvc(self.k)):
                self.setto("e")

    def step1c(self):
        """step1c() turns terminal y to i when there is another vowel in the stem."""
        if (self.ends("y") and self.vowelinstem()):
            self.b = self.b[:self.k] + 'i' + self.b[self.k+1:]

    def step2(self):
        """step2() maps double suffices to single ones.
        so -ization ( = -ize plus -ation) maps to -ize etc. note that the
        string before the suffix must give m() > 0.
        """
        if self.b[self.k - 1] == 'a':
            if self.ends("ational"):   self.r("ate")
            elif self.ends("tional"):  self.r("tion")
        elif self.b[self.k - 1] == 'c':
            if self.ends("enci"):      self.r("ence")
            elif self.ends("anci"):    self.r("ance")
        elif self.b[self.k - 1] == 'e':
            if self.ends("izer"):      self.r("ize")
        elif self.b[self.k - 1] == 'l':
            if self.ends("bli"):       self.r("ble") # --DEPARTURE--
            # To match the published algorithm, replace this phrase with
            #   if self.ends("abli"):      self.r("able")
            elif self.ends("alli"):    self.r("al")
            elif self.ends("entli"):   self.r("ent")
            elif self.ends("eli"):     self.r("e")
            elif self.ends("ousli"):   self.r("ous")
        elif self.b[self.k - 1] == 'o':
            if self.ends("ization"):   self.r("ize")
            elif self.ends("ation"):   self.r("ate")
            elif self.ends("ator"):    self.r("ate")
        elif self.b[self.k - 1] == 's':
            if self.ends("alism"):     self.r("al")
            elif self.ends("iveness"): self.r("ive")
            elif self.ends("fulness"): self.r("ful")
            elif self.ends("ousness"): self.r("ous")
        elif self.b[self.k - 1] == 't':
            if self.ends("aliti"):     self.r("al")
            elif self.ends("iviti"):   self.r("ive")
            elif self.ends("biliti"):  self.r("ble")
        elif self.b[self.k - 1] == 'g': # --DEPARTURE--
            if self.ends("logi"):      self.r("log")
        # To match the published algorithm, delete this phrase

    def step3(self):
        """step3() dels with -ic-, -full, -ness etc. similar strategy to step2."""
        if self.b[self.k] == 'e':
            if self.ends("icate"):     self.r("ic")
            elif self.ends("ative"):   self.r("")
            elif self.ends("alize"):   self.r("al")
        elif self.b[self.k] == 'i':
            if self.ends("iciti"):     self.r("ic")
        elif self.b[self.k] == 'l':
            if self.ends("ical"):      self.r("ic")
            elif self.ends("ful"):     self.r("")
        elif self.b[self.k] == 's':
            if self.ends("ness"):      self.r("")

    def step4(self):
        """step4() takes off -ant, -ence etc., in context <c>vcvc<v>."""
        if self.b[self.k - 1] == 'a':
            if self.ends("al"): pass
            else: return
        elif self.b[self.k - 1] == 'c':
            if self.ends("ance"): pass
            elif self.ends("ence"): pass
            else: return
        elif self.b[self.k - 1] == 'e':
            if self.ends("er"): pass
            else: return
        elif self.b[self.k - 1] == 'i':
            if self.ends("ic"): pass
            else: return
        elif self.b[self.k - 1] == 'l':
            if self.ends("able"): pass
            elif self.ends("ible"): pass
            else: return
        elif self.b[self.k - 1] == 'n':
            if self.ends("ant"): pass
            elif self.ends("ement"): pass
            elif self.ends("ment"): pass
            elif self.ends("ent"): pass
            else: return
        elif self.b[self.k - 1] == 'o':
            if self.ends("ion") and (self.b[self.j] == 's' or self.b[self.j] == 't'): pass
            elif self.ends("ou"): pass
            # takes care of -ous
            else: return
        elif self.b[self.k - 1] == 's':
            if self.ends("ism"): pass
            else: return
        elif self.b[self.k - 1] == 't':
            if self.ends("ate"): pass
            elif self.ends("iti"): pass
            else: return
        elif self.b[self.k - 1] == 'u':
            if self.ends("ous"): pass
            else: return
        elif self.b[self.k - 1] == 'v':
            if self.ends("ive"): pass
            else: return
        elif self.b[self.k - 1] == 'z':
            if self.ends("ize"): pass
            else: return
        else:
            return
        if self.m() > 1:
            self.k = self.j

    def step5(self):
        """step5() removes a final -e if m() > 1, and changes -ll to -l if
        m() > 1.
        """
        self.j = self.k
        if self.b[self.k] == 'e':
            a = self.m()
            if a > 1 or (a == 1 and not self.cvc(self.k-1)):
                self.k = self.k - 1
        if self.b[self.k] == 'l' and self.doublec(self.k) and self.m() > 1:
            self.k = self.k -1

    def stem(self, p, i, j):
        """In stem(p,i,j), p is a char pointer, and the string to be stemmed
        is from p[i] to p[j] inclusive. Typically i is zero and j is the
        offset to the last character of a string, (p[j+1] == '\0'). The
        stemmer adjusts the characters p[i] ... p[j] and returns the new
        end-point of the string, k. Stemming never increases word length, so
        i <= k <= j. To turn the stemmer into a module, declare 'stem' as
        extern, and delete the remainder of this file.
        """
        # copy the parameters into statics
        self.b = p
        self.k = j
        self.k0 = i
        if self.k <= self.k0 + 1:
            return self.b # --DEPARTURE--

        # With this line, strings of length 1 or 2 don't go through the
        # stemming process, although no mention is made of this in the
        # published algorithm. Remove the line to match the published
        # algorithm.

        self.step1ab()
        self.step1c()
        self.step2()
        self.step3()
        self.step4()
        self.step5()
        return self.b[self.k0:self.k+1]


In [17]:
stop_words = stopwords.words('english')

def stop_word_removal(text, stop_word_corpus, punct_str):
    clean_text = ''.join([word.lower() for word in text.split() if word.lower()
                 not in stop_word_corpus]).replace('\n',' ')
    return clean_text.translate(str.maketrans('','', punct_str))


# Change the key values from double list to string
# We can use the double join for conversion

In [81]:
for i in range(len(d.keys())):
    Values=("',  '".join(" ".join(l)for l in d.values()))

Final=list(Values.split("',  '"))


# For query Processing

In [94]:
qeury=input("Enter the Bag of words you are looking for ")
query_List=list(qeury.split())


Enter the query you are looking for Computers are the coded machines


In [95]:
query_clean= [stop_word_removal(headline,stop_words,string.punctuation)
                for headline in query_List]

In [33]:
cleaned_word = [stop_word_removal(headline,stop_words,string.punctuation)
                for headline in Final]

# Stemming The cleaned words

In [85]:
p=PorterStemmer()
stemmedWord=[]
for words in cleaned_word:
    stw=p.stem(words, 0,len(words)-1)
    stemmedWord.append(stw)
Stemmed_String=' '.join(stemmedWord)


# Computing TF

In [43]:
def term_freq(document_list):
    
    '''
    Function that returns the term frequency of given documents
    
    Parameters
    ----------
    document_list: list or array containing text in string format
    
    Returns
    -------
    {document: vector} : dictionary containing the original string input as the key
    and the term frequency for the given article
    '''
    
    combined_string = ' '.join(document_list)
    word_set = [word for word in set(combined_string.split()) if len(word) > 1]

    filtered_docs = []
    for document in document_list:
        filtered_docs.append(' '.join([word for word in document.split() if len(word) > 1]))

    doc_word_count = []
    for document in filtered_docs:
        doc_word_count.append([document.split().count(word)/len(document.split(', ')) for word in word_set])

    return {doc:vector for doc,vector in zip(filtered_docs,doc_word_count)}

In [44]:
def word_list(document_list):
    
    '''
    Function that returns the unique words for a given list of strings.
    
    Parameters
    ----------
    document_list: list or array containing text in string format
    
    Returns
    -------
    [words] : list containing a unique set of words
    '''
    
    combined_string = ' '.join(document_list)
    return [word for word in set(combined_string.split()) if len(word) > 1]


In [45]:
tf_df = pd.DataFrame(data = term_freq(stemmedWord).values(),
                  columns = word_list(stemmedWord),
                  index = term_freq(stemmedWord).keys())

tf_df

,cacm0033html131,919611,introductory5998,incomplete,6916316,environment,2546271998,pm12236491,19015168,gleissner,...,2cacm1437,4383643543,7206474,computeroriented,1601611,cacm0250html998,4265426,1959kaner900,where7523,system9372
html pre preliminary reportinternational algebraic language cacm december 1958 perlis samelsonk ca581203 jb march 22 1978 828 pm 10051 12351 16451 151 151 151 20551 21051 21451 198251 39851 64251 66951 161 161 161 161 161 161 161 161 161 161 16561 19661 19661 127361 188361 32461 4361 5361 9161 41061 318461 pre html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
html pre extraction roots repeated subtractions digital computers cacm december 1958 sugai ca581202 jb march 22 1978 829 pm 252 252 252 pre html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
html pre techniques department matrix program schemes cacm december 1958 friedman ca581201 jb march 22 1978 830 pm 353 353 353 pre html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
html pre glossary computer engineering programming terminology cacm november 1958 ca581103 jb march 22 1978 832 pm 454 454 454 pre html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
html pre two squareroot approximations cacm november 1958 wadey ca581102 jb march 22 1978 833 pm 555 555 555 pre html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
html pre use computers inspection procedures cacm november 1958 muller ca581101 jb march 22 1978 833 pm 656 656 656 47756 666 pre html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
html pre glossary computer engineering programming terminology cacm october 1958 ca581003 jb march 22 1978 835 pm 757 757 757 pre html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
html pre equivalence transformation program schemes cacm october 1958 friedman ca581002 jb march 22 1978 836 pm 858 858 858 pre html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
html pre proposal uncol cacm october 1958 conway ca581001 jb march 22 1978 837 pm 959 959 959 pre html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
html pre glossary computer engineering programming terminology cacm september 1958 ca580903 jb march 22 1978 839 pm 10510 10510 10510 pre html,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
def inverse_doc_freq(document_list,unique_words):
    
    '''
   
    '''
    
    idf_dict = {}
    
    for word in unique_words:
        temp_word_list = []
        for sentence in document_list:
            temp_word_list.append(word in sentence)
        idf_dict[word] = temp_word_list.count(True)+1
        
    idf_list = []

    for value in idf_dict.values():
        idf_list.append(math.log((len(tf_df)+1)/value)+1)
        
    return idf_list


In [47]:
def inverse_doc_freq(document_list,unique_words):
    
    
    idf_dict = {}
    
    for word in unique_words:
        temp_word_list = []
        for sentence in document_list:
            temp_word_list.append(word in sentence)
        idf_dict[word] = temp_word_list.count(True)+1
        
    idf_list = []

    for value in idf_dict.values():
        idf_list.append(math.log((len(tf_df)+1)/value)+1)
        
    return idf_list


In [48]:
idf_array = inverse_doc_freq(tf_df.index,tf_df.columns)


In [92]:
def tf_idf(term_freq_data,inverse_doc_freq_data):
    
    tf_idf_vector = []
    multiplied_vector = np.array(term_freq_data) * np.array(inverse_doc_freq_data)
    
    for array in multiplied_vector:
        tf_idf_vector.append(array/(sum(array**2))**.5)
        
    return tf_idf_vector

# Computing TF IDF for a term in document

In [124]:
tfidf_df = pd.DataFrame(data = tf_idf(tf_df.to_numpy(), np.array(idf_array)),
             columns = word_list(stemmedWord),
             index = term_freq(stemmedWord).keys())


tfidf_df

,cacm0033html131,919611,introductory5998,incomplete,6916316,environment,2546271998,pm12236491,19015168,gleissner,...,2cacm1437,4383643543,7206474,computeroriented,1601611,cacm0250html998,4265426,1959kaner900,where7523,system9372
html pre preliminary reportinternational algebraic language cacm december 1958 perlis samelsonk ca581203 jb march 22 1978 828 pm 10051 12351 16451 151 151 151 20551 21051 21451 198251 39851 64251 66951 161 161 161 161 161 161 161 161 161 161 16561 19661 19661 127361 188361 32461 4361 5361 9161 41061 318461 pre html,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
html pre extraction roots repeated subtractions digital computers cacm december 1958 sugai ca581202 jb march 22 1978 829 pm 252 252 252 pre html,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
html pre techniques department matrix program schemes cacm december 1958 friedman ca581201 jb march 22 1978 830 pm 353 353 353 pre html,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
html pre glossary computer engineering programming terminology cacm november 1958 ca581103 jb march 22 1978 832 pm 454 454 454 pre html,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
html pre two squareroot approximations cacm november 1958 wadey ca581102 jb march 22 1978 833 pm 555 555 555 pre html,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
html pre use computers inspection procedures cacm november 1958 muller ca581101 jb march 22 1978 833 pm 656 656 656 47756 666 pre html,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
html pre glossary computer engineering programming terminology cacm october 1958 ca581003 jb march 22 1978 835 pm 757 757 757 pre html,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
html pre equivalence transformation program schemes cacm october 1958 friedman ca581002 jb march 22 1978 836 pm 858 858 858 pre html,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
html pre proposal uncol cacm october 1958 conway ca581001 jb march 22 1978 837 pm 959 959 959 pre html,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
html pre glossary computer engineering programming terminology cacm september 1958 ca580903 jb march 22 1978 839 pm 10510 10510 10510 pre html,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [100]:
from sklearn.metrics.pairwise import cosine_similarity

# Cosine Similarity for the term document 

In [64]:
cosine_similarity(tfidf_df,)

array([[1.        , 0.03497576, 0.03685176, ..., 0.00718987, 0.        ,
        0.        ],
       [0.03497576, 1.        , 0.09089789, ..., 0.0177344 , 0.        ,
        0.        ],
       [0.03685176, 0.09089789, 1.        , ..., 0.01868563, 0.        ,
        0.        ],
       ...,
       [0.00718987, 0.0177344 , 0.01868563, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [120]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')
tfidf_wm = tfidfvectorizer.fit_transform(query_clean)
tfidf_df_q = pd.DataFrame(tfidf_wm.toarray(), columns=tfidfvectorizer.get_feature_names())


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# TF-IDF of the query to be searched

In [121]:
tfidf_df_q 

,coded,computers,machines
0,0.0,1.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0



# cosine similarity between the query and the document

In [123]:
results=cosine_similarity(tfidf_df,tfidf_df_q)
results

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

# Retrieving The First 20 documnets for a given query 

In [118]:
for i in results.argsort()[-20:][::-1]:
    print(tfidf_df.iloc[i,0],"--",tfidf_df.iloc[i,1])

0    0.0
1    0.0
2    0.0
3    1.0
4    0.0
Name: coded, dtype: float64 -- 0    1.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: computers, dtype: float64
0    0.0
1    0.0
2    0.0
4    0.0
3    1.0
Name: coded, dtype: float64 -- 0    1.0
1    0.0
2    0.0
4    0.0
3    0.0
Name: computers, dtype: float64
0    0.0
1    0.0
2    0.0
3    1.0
4    0.0
Name: coded, dtype: float64 -- 0    1.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: computers, dtype: float64
0    0.0
1    0.0
2    0.0
3    1.0
4    0.0
Name: coded, dtype: float64 -- 0    1.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: computers, dtype: float64
1    0.0
2    0.0
3    1.0
4    0.0
0    0.0
Name: coded, dtype: float64 -- 1    0.0
2    0.0
3    0.0
4    0.0
0    1.0
Name: computers, dtype: float64


In [52]:
#print metrics on corpus

In [53]:
print ('Processing Start Time: %.2d:%.2d' % (t2.tm_hour, t2.tm_min))
print ("Documents %i" % documents)
print ("Tokens %i" % tokens)
print ("Terms %i" % terms)
t2=time.localtime()
print ('Processing End Time: %.2d:%.2d' % (t2.tm_hour, t2.tm_min))


Processing Start Time: 21:29
Documents 1144
Tokens 26545
Terms 9608
Processing End Time: 21:48
